In [5]:
import pybaobabdt
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, roc_curve
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import time

#%% Read the data
train = pd.read_csv("../data/adj_imp_train.csv")
test = pd.read_csv("../data/adj_imp_test.csv")
# train = pd.read_csv("..\\bankruptcy data\\training_knn.csv",index_col=0)
# test = pd.read_csv("..\\bankruptcy data\\testing_knn.csv",index_col=0)
# train = pd.read_csv("..\\bankruptcy data\\smote_train.csv")
# test = pd.read_csv("..\\bankruptcy data\\adj_imp_test.csv")

# train = pd.read_csv("..\\bankruptcy data\\pca_train.csv")
# test = pd.read_csv("..\\bankruptcy data\\pca_test.csv")

train_X = train.iloc[:,:-1]/10
train_y = train.iloc[:,-1]
train_y.apply(str)

test_X = test.iloc[:,:-1]/10
test_y = test.iloc[:,-1]
test_y.apply(str)

#%% Model Buliding
features =list(train.columns)

model = tree.DecisionTreeClassifier(criterion='entropy',random_state=0,min_samples_leaf=40, max_depth=6)
# model = tree.DecisionTreeClassifier(criterion='gini',random_state=0,min_samples_leaf=37)
tic = time.time()
model.fit(train_X,list(train_y))
toc = time.time()
# fig = plt.figure(dpi=500)
# ax1 = fig.add_subplot(111)
# pybaobabdt.drawTree(model, size=10, dpi=500, features=features[:-1], colormap='Set1',ax=ax1)
# fig.savefig('C:\\Users\\L\Desktop\\sta\\FTE4560\\project-1\\report\\Decision Tree.png', format='png', dpi=300, transparent=True)
#%% Model report test set
# predict = model.predict(test_X)
tic = time.time()
score = model.predict_proba(test_X)[:,1]
toc = time.time()
predict = score >= 0.3
auc = roc_auc_score(test_y, score)

fpr, tpr, _ = roc_curve(test_y, score)

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.plot(fpr, tpr, color='darkorange',
         label='ROC curve (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for testing set')
plt.legend(loc="lower right")

print("auc:", auc)
print(classification_report(test_y, predict))

#%% Model report train set
# predict = model.predict(train_X)
score = model.predict_proba(train_X)[:,1]
predict = score >= 0.3
auc = roc_auc_score(train_y, score)

fpr, tpr, _ = roc_curve(train_y, score)


plt.subplot(122)
plt.plot(fpr, tpr, color='darkorange',
         label='ROC curve (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for training set')
plt.legend(loc="lower right")
plt.show()

print("auc:", auc)
print(classification_report(train_y, predict))


#%% Change the hyperparameter

auc_list=[] 
precision_list=[]
recall_list=[]
f1score_list=[]
for i in range(1,20): 
    model = tree.DecisionTreeClassifier(criterion='entropy',random_state=0,max_depth=i)
    model.fit(train_X,train_y)
    score = model.predict_proba(test_X)[:,1]
    predict = score >= 0.33
    auc=roc_auc_score(test_y, score)
    precision = precision_score(test_y, predict)
    recall = recall_score(test_y,predict)
    f1score = f1_score(test_y,predict)
    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)
    f1score_list.append(f1score)

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.plot(range(1,20),auc_list,label="auc")
plt.plot(range(1,20),precision_list,label="precision")
plt.plot(range(1,20),recall_list,label="recall")
plt.plot(range(1,20),f1score_list,label="f1 score")
plt.legend()
plt.xlabel('max depth')
plt.title('Entropy')
plt.ylabel('indicator')


auc_list=[] 
precision_list=[]
recall_list=[]
f1score_list=[]
for i in range(1,20): 
    model = tree.DecisionTreeClassifier(criterion='gini',random_state=0,max_depth=i)
    model.fit(train_X,train_y)
    score = model.predict_proba(test_X)[:,1]
    predict = score >= 0.33
    auc=roc_auc_score(test_y, score)
    precision = precision_score(test_y, predict)
    recall = recall_score(test_y,predict)
    f1score = f1_score(test_y,predict)
    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)
    f1score_list.append(f1score)

plt.subplot(122)
plt.plot(range(1,20),auc_list,label="auc")
plt.plot(range(1,20),precision_list,label="precision")
plt.plot(range(1,20),recall_list,label="recall")
plt.plot(range(1,20),f1score_list,label="f1 score")
plt.legend()
plt.xlabel('max depth')
plt.title('Gini')
plt.ylabel('indicator')
plt.show()

#%% Change the hyperparameter

auc_list=[] 
precision_list=[]
recall_list=[]
f1score_list=[]
for i in range(1,100): 
    model = tree.DecisionTreeClassifier(criterion='entropy',random_state=0,max_depth=6,min_samples_leaf=i)
    model.fit(train_X,train_y)
    score = model.predict_proba(test_X)[:,1]
    predict = score >= 0.33
    auc=roc_auc_score(test_y, score)
    precision = precision_score(test_y, predict)
    recall = recall_score(test_y,predict)
    f1score = f1_score(test_y,predict)
    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)
    f1score_list.append(f1score)
    
plt.plot(range(1,100),auc_list,label="auc")
plt.plot(range(1,100),precision_list,label="precision")
plt.plot(range(1,100),recall_list,label="recall")
plt.plot(range(1,100),f1score_list,label="f1 score")
plt.legend()
plt.xlabel('min samples leaf')
plt.ylabel('indicator')
plt.show()

ModuleNotFoundError: No module named 'pygraphviz'